In this example, we will be accessing data from a managed Iceberg table with Tasty Bytes restaurant reviews stored in a data lake. We will write feature engineering code in Notebooks using pandas on Snowflake. pandas on Snowflake offers familiar API and syntax for data cleaning and transformation that is more flexible than convenient than SQL for programmatic specification.

Demo Overview:
1. **Connect Iceberg table with pandas on Snowflake**: Read Iceberg table from Snowflake via `read_snowflake`
2. **Data Profiling and Cleaning**: Perform exploratory analysis to profile data characteristics and clean data
3. **Feature Engineering**: Perform feature engineering tasks with pandas on Snowflake, including using Cortex LLM functions
4. **Save as Dynamic Iceberg Table**: Save resulting table with new feature as a Dynamic Iceberg Table 
5. **Automatic Pipeline Refresh with Dynamic Iceberg Table**: Changes to source iceberg table automatically refreshes the dynamic iceberg table.

Required packages: `modin`, `snowflake-ml-python` 

## Connect Iceberg table with pandas on Snowflake

We have a Iceberg table `RESTAURANT_REVIEWS_ICEBERG` that we'll be working with. You can learn more about how you can set up your Iceberg table in [the documentation](https://docs.snowflake.com/en/user-guide/tutorials/create-your-first-iceberg-table).

Now let's use [pandas on Snowflake](https://docs.snowflake.com/developer-guide/snowpark/python/pandas-on-snowflake) to connect to the iceberg table as a dataframe we can work with.

In [ ]:
import snowflake.snowpark.modin.plugin
import modin.pandas as pd

from snowflake.snowpark.session import Session
session = get_active_session()

In [ ]:
df = pd.read_snowflake("RESTAURANT_REVIEWS_ICEBERG")

## Data Profiling and Cleaning
Let's take a look at our data and overall statistics.

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df = df.drop("SOURCE", axis=1)
df.head()

Let's look at the number of records for `PRIMARY_CITY`.

In [ ]:
df["PRIMARY_CITY"].value_counts()

## Feature Engineering
Tasty Bytes is a global food truck operation. You can see that there are many cities across the globe that is listed. To simplify our analysis of geographical impact on restaurant reviews, we want to map cities to a smaller set of geographical regions. To do this, we create a new dataframe with the unique cities values from the source dataframe.

In [ ]:
city_mapping = pd.DataFrame(df["PRIMARY_CITY"].unique(),columns=["PRIMARY_CITY"])
city_mapping

Next, to clean up the data further, we want to to classify cities into a smaller number of regions. We can do that using Snowflake's Cortex LLM functions [CLASSIFY_TEXT](https://docs.snowflake.com/en/sql-reference/functions/classify_text-snowflake-cortex). You can use Snowflake Cortex LLM functions via the Snowpark pandas apply function, see examples [here](https://docs.snowflake.com/en/developer-guide/snowpark/python/pandas-on-snowflake#using-snowflake-cortex-llm-functions-with-snowpark-pandas).

In [ ]:
from snowflake.cortex import ClassifyText
city_mapping["REGION"] = city_mapping.apply(ClassifyText, categories=["Asia","Europe","Americas","Africa"])

In [ ]:
# Extract Region label from the mapping
city_mapping["REGION_LABEL"] = city_mapping["REGION"].apply(lambda x: x.get('label'))
city_mapping[['PRIMARY_CITY','REGION_LABEL']]

In [ ]:
# Create a mapping dictionary
city_to_region = dict(zip(city_mapping['PRIMARY_CITY'], city_mapping['REGION_LABEL']))

In [ ]:
# Map the REGION_LABEL into the original DataFrame
feature_df = df
feature_df['REGION'] = feature_df['PRIMARY_CITY'].map(city_to_region)

In [ ]:
import altair as alt

# Convert the value_counts to a DataFrame for Altair
region_counts = feature_df['REGION'].value_counts().reset_index()
region_counts.columns = ['REGION', 'COUNT']

# Create the horizontal bar chart with text labels
chart = alt.Chart(region_counts).mark_bar().encode(
    x=alt.X('COUNT:Q', title='Number of Reviews'),
    y=alt.Y('REGION:N', title='Region', sort='-x'),
    color=alt.Color('REGION:N', legend=None)
).properties(
    title='Number of Reviews by Region',
    width=600,
    height=400
)

chart

We can see from the dataframe preview of `feature_df` that we now have a new feature `REGION` in our resulting dataframe.

In [ ]:
feature_df.head()

## Save as Iceberg Table

We want to save the resulting table with new feature as a Iceberg Table. Here is our [iceberg configuration](https://docs.snowflake.com/en/sql-reference/sql/create-iceberg-table-snowflake):

In [ ]:
iceberg_config = {
    "external_volume": "ICEBERG_EXTERNAL_VOLUME",
    "catalog": "SNOWFLAKE",
    "base_location": "RESTAURANT_REVIEWS_ICEBERG/"
}

Now we can save this back as an Iceberg table using Snowpark pandas's `to_iceberg` operation: 

In [ ]:
feature_df.to_iceberg("RESTAURANT_REVIEWS_FEATURES_ICEBERG", iceberg_config = iceberg_config)

## Save as dynamic Apache Iceberg tables

What if my source data is changing as new restaurant reviews stream in regularly and we want to make sure that our downstream tables are refreshed as new data come in?
To achieve this, we can save the table as a [dynamic iceberg table](https://docs.snowflake.com/en/user-guide/dynamic-tables-create-iceberg). Automated data transformation with dynamic Iceberg tables uses declarative SQL to define the desired end state without managing intermediary steps. Snowflake handles orchestration, scheduling, and refreshing data transformations based on your specified data freshness targets.

For the purpose of the demo here, we specify our data freshness target lag as one minute. Note that I specified `refresh_mode="incremental"` so that Snowflake analyzes the query and calculates and merges back the changes since the last refresh, to avoid a refresh on the full table unless necessary.


In [ ]:
feature_df.to_dynamic_table("RESTAURANT_REVIEWS_FEATURES_DYNAMIC_ICEBERG_TABLE",index=None, lag = "1minutes", warehouse = "ICEBERG_TUTORIAL_WH",iceberg_config = iceberg_config, refresh_mode="incremental")

## Automatic Pipeline Refresh with Dynamic Iceberg Table

Now, let's insert 10 new rows to our original iceberg table source. Changes to source table should automatically refreshes the dynamic iceberg table with our new features.

In [ ]:
INSERT INTO RESTAURANT_REVIEWS_ICEBERG (REVIEW_ID, ORDER_ID, TRUCK_ID, LANGUAGE, SOURCE, REVIEW, PRIMARY_CITY, CUSTOMER_ID, DATE, TRUCK_BRAND_NAME) VALUES
(81257, 367914035, 326, 'en', NULL, 'The "Revenge of the Curds" truck in Rio de Janeiro is a poutine paradise! The Classic, Mother''s Favorite, and The Kitchen Sink were all exceptional. Each dish was a unique blend of flavors, with the perfect balance of crispy fries, rich gravy, and squeaky cheese curds. The bottled water and soda were a refreshing complement to the meal. The truck''s staff was friendly and efficient, making the experience even better. If you''re a poutine lover, this is a must-visit!', 'Rio de Janeiro', NULL, '2024-06-01', 'Revenge of the Curds'),
(81260, 367920093, 330, 'en', NULL, 'The Hot Ham & Cheese sandwich at Better Off Bread in Rio de Janeiro proved to be a disappointing culinary experience. The bread was distressingly limp and stale, the ham was overcooked and dry, and the cheese was not only scant but also lacking in flavor. The only saving grace was the generous helping of mayonnaise. Regrettably, the subpar quality of the ingredients negated any potential for a satisfying sandwich. It is, therefore, not a sandwich I would recommend to anyone.', 'Rio de Janeiro', NULL, '2024-06-01', 'Better Off Bread'),
(4, 10869030, 59, 'en', NULL, ' The Combination Curry from Nani''s Kitchen was a disappointing experience. The curry lacked depth in flavor, tasting more like a watered-down version of what it should have been. The vegetables were overcooked and mushy, and the promised variety of ingredients was not evident. The portion size was also smaller than expected for the price. Overall, not the authentic Indian cuisine experience I was hoping for in Boston.', 'Boston', NULL, '2024-06-01', 'Nani''s Kitchen'),
(114726, 435950617, 445, 'en', NULL, 'Disappointment was the main course at "Not the Wurst Hot Dogs". The Chicago Dogs, ordered twice in hopes of a better experience, were lackluster. The bun was soggy, the toppings were unbalanced, and the hot dog itself was bland. The overall taste was underwhelming, failing to deliver the authentic Chicago Dog experience. Cape Town deserves better hot dogs than this.', 'Cape Town', NULL, '2024-06-01', 'Not the Wurst Hot Dogs'),
(27861, 63960366, 182, 'en', NULL, 'I am extremely disappointed with the fried pickles I ordered from Smoky BBQ in Warsaw. Not only were they woefully undersized, but the breading was limp and greasy, detracting from the overall flavor. The pickles themselves were bland and uninspiring. This is unacceptable for a truck touted as a BBQ specialist. I would not recommend this truck to anyone seeking a satisfying fried pickle experience.', 'Warsaw', NULL, '2024-06-01', 'Smoky BBQ'),
(100062, 408472251, 394, 'en', NULL, 'The Lobster Mac & Cheese from The Mac Shack was a disappointing experience. The lobster was overcooked and tough, lacking the tender, succulent texture one expects. The cheese sauce was bland and lacked the rich, creamy consistency that makes mac & cheese a comfort food favorite. The Ice Tea was watered down and lackluster. For a food truck in Madrid, I expected more flavor and quality. Unfortunately, The Mac Shack did not deliver.', 'Madrid', NULL, '2024-06-01', 'The Mac Shack'),
(91122, 392630366, 363, 'en', NULL, 'The Veggie Taco Bowl, Lean Burrito Bowl, and Two Taco Combo Plate from Guac n'' Roll were disappointing. The veggies were overcooked, lacking the fresh crunch expected in a taco bowl. The lean burrito bowl was bland, missing the flavorful kick that usually comes with Mexican cuisine. The tacos in the combo plate were small and underwhelming, not living up to the ''Guac n'' Roll'' name as the guacamole was scarce. Overall, not a satisfying experience from this Hamburg-based truck.', 'Hamburg', NULL, '2024-06-01', 'Guac n'' Roll'),
(21764, 53568604, 162, 'en', NULL, 'The Cheeky Greek truck in Paris offered a satisfying meal with The King Combo and Greek Salad. The gyro meat was flavorful and tender, complemented well by the fresh vegetables and creamy tzatziki sauce. The Greek Salad was a refreshing side, with crisp lettuce, juicy tomatoes, and tangy feta cheese. The Ice Tea was a nice touch, cooling down the spicy notes of the combo. However, the pita bread could have been warmer and softer. Overall, it was a decent experience, but there''s still room for improvement in some aspects.', 'Paris', NULL, '2024-06-01', 'Cheeky Greek'),
(104348, 417184811, 417, 'en', NULL, 'A Gyro Plate and Greek Salad combo was served from The Cheeky Greek truck in Barcelona. The flavorful gyro meat was appreciated, but the pita bread lacked moisture. The Greek salad was crisp and fresh, enhancing the meal''s enjoyment. A bottle of water was an appreciated refreshment on a warm day. The experience was mostly satisfactory, with potential for enhancement, specifically in the pita bread category.', 'Barcelona', NULL, '2024-06-01', 'Cheeky Greek'),
(27865, 63972722, 194, 'en', NULL, 'Nani''s Kitchen delivered a satisfying Indian feast, exceeding my expectations. The lean chicken tikka masala was tender and flavorful, with the perfect balance of spices and creamy sauce. The combination curry offered a diverse range of textures and aromas, complementing the tikka masala perfectly. The portions were generous, and the service was prompt. Overall, I highly recommend Nani''s Kitchen for anyone seeking authentic Indian cuisine in Warsaw.', 'Warsaw', NULL, '2024-06-01', 'Nani''s Kitchen');


After running the insert query, we can look at the dynamic iceberg table created `RESTAURANT_REVIEWS_FEATURES_DYNAMIC_ICEBERG` in the Snowsight Object Explorer and see the feature table has been automatically refreshed. With incremental refresh, only the inserted 10 rows needs to be updated, so we see that there was 10 rows changed. This is much cheaper and faster than trying to rerun the pipeline on the full table.

In [ ]:
SELECT * FROM RESTAURANT_REVIEWS_FEATURES_DYNAMIC_ICEBERG_TABLE WHERE REVIEW_ID IN (81257, 81260, 4, 114726, 27861, 100062, 91122, 21764, 104348, 27865)

## Teardown 

In [ ]:
DELETE FROM RESTAURANT_REVIEWS_ICEBERG
WHERE REVIEW_ID IN (
    81257, 81260, 4, 114726, 27861, 100062, 91122, 21764, 104348, 27865
);

In [ ]:
DROP TABLE IF EXISTS RESTAURANT_REVIEWS_FEATURES_ICEBERG;
DROP TABLE IF EXISTS RESTAURANT_REVIEWS_FEATURES_DYNAMIC_ICEBERG_TABLE;